In [2]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import scipy.stats as st
import seaborn as sns
import pandas as pd

In [3]:
# пакет для работы с множественным тестированием гипотез
from statsmodels.stats.multitest import multipletests

# Сравнение качества классификаторов

Дано: AUC различных версий алгоритма C4.5, посчитанный на 15 датасетах.

*Demsar J. (2006). Statistical Comparisons of Classifiers over Multiple Data Sets. Journal of Machine Learning Research, 7, 1–30.* 

## Рассмотрим данные

In [4]:
data = pd.read_csv('./auc.txt', delimiter='\t')

In [7]:
data

,Dataset,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898
5,iris,0.936,0.931,0.916,0.931
6,liver disorders,0.661,0.668,0.609,0.685
7,lung cancer,0.583,0.583,0.563,0.625
8,lymphography,0.775,0.838,0.866,0.875
9,mushroom,1.000,1.000,1.000,1.000


In [6]:
data.shape

(14, 5)

Между какими из версий есть значимые различия?

Сравним все значения критерия качества попарно с помощью критерия знаковых рангов Уилкоксона (```st.wilcoxon```):

In [8]:
p_values= []
for i in range(1, 5):
    for j in range(i+1, 5):
        test_result = st.wilcoxon(data.iloc[:,i].values, data.iloc[:,j].values)
        p_values += [test_result[1]]        
        print(data.columns[i],data.columns[j], p_values[-1])
                                                               

C4.5 C4.5+m 0.01075713311978963
C4.5 C4.5+cf 0.861262330095348
C4.5 C4.5+m+cf 0.015906444101703374
C4.5+m C4.5+cf 0.046332729793395394
C4.5+m C4.5+m+cf 0.3278256758446406
C4.5+cf C4.5+m+cf 0.022909099354356588


Попробуем разные поправки.

In [12]:
for k in ['bonferroni', 'holm', 'fdr_bh', 'fdr_by']:
    result = multipletests(p_values, method = k)
    print(k)
    print(result[0])
    print(result[1])
    print('-'*30)

bonferroni
[False False False False False False]
[0.0645428  1.         0.09543866 0.27799638 1.         0.1374546 ]
------------------------------
holm
[False False False False False False]
[0.0645428  0.86126233 0.07953222 0.13899819 0.65565135 0.0916364 ]
------------------------------
fdr_bh
[ True False  True False False  True]
[0.0458182  0.86126233 0.0458182  0.06949909 0.39339081 0.0458182 ]
------------------------------
fdr_by
[False False False False False False]
[0.11225459 1.         0.11225459 0.17027278 0.96380749 0.11225459]
------------------------------
